<div style="display: flex; justify-content: space-between; align-items: center;">
    <div style="text-align: left; flex: 4;">
        <strong>Author:</strong> Amirhossein Heydari — 
        📧 <a href="mailto:amirhosseinheydari78@gmail.com">amirhosseinheydari78@gmail.com</a> — 
        🐙 <a href="https://github.com/mr-pylin/media-processing-workshop" target="_blank" rel="noopener">github.com/mr-pylin</a>
    </div>
    <div style="display: flex; justify-content: flex-end; flex: 1; gap: 8px; align-items: center; padding: 0;">
        <a href="https://opencv.org/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/opencv/logo/OpenCV_logo_no_text-1.svg"
                 alt="OpenCV Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://pillow.readthedocs.io/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/pillow/logo/pillow-logo-248x250.png"
                 alt="PIL Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scikit-image.org/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/scikit-image/logo/logo.png"
                 alt="scikit-image Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scipy.org/" target="_blank" rel="noopener noreferrer">
            <img src="../assets/images/libraries/scipy/logo/logo.svg"
                 alt="SciPy Logo"
                 style="max-height: 48px; width: auto;">
        </a>
    </div>
</div>
<hr>


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Prepare Images](#toc2_)    
  - [Binary Images](#toc2_1_)    
  - [Grayscale Images](#toc2_2_)    
  - [RGB Images](#toc2_3_)    
- [Morphological Processing](#toc3_)    
  - [Structuring Elements](#toc3_1_)    
    - [Square](#toc3_1_1_)    
    - [Disk (Circular)](#toc3_1_2_)    
    - [Diamond](#toc3_1_3_)    
    - [Octagon](#toc3_1_4_)    
    - [Line](#toc3_1_5_)    
    - [Cross (Plus)](#toc3_1_6_)    
    - [Rectangle](#toc3_1_7_)    
  - [Morphological Operators](#toc3_2_)    
    - [Erosion ($\ominus$)](#toc3_2_1_)    
      - [Binary Erosion](#toc3_2_1_1_)    
        - [Using OpenCV](#toc3_2_1_1_1_)    
        - [Using Scikit-Image](#toc3_2_1_1_2_)    
        - [Using SciPy](#toc3_2_1_1_3_)    
      - [Grayscale Erosion](#toc3_2_1_2_)    
    - [Dilation ($\oplus$)](#toc3_2_2_)    
      - [Binary Dilation](#toc3_2_2_1_)    
      - [Grayscale Dilation](#toc3_2_2_2_)    
    - [Opening ($\circ$)](#toc3_2_3_)    
      - [Binary Opening](#toc3_2_3_1_)    
      - [Grayscale Opening](#toc3_2_3_2_)    
    - [Closing ($\bullet$)](#toc3_2_4_)    
      - [Binary Closing](#toc3_2_4_1_)    
      - [Grayscale Closing](#toc3_2_4_2_)    
    - [Gradient ($\nabla$)](#toc3_2_5_)    
      - [Binary Gradient](#toc3_2_5_1_)    
      - [Grayscale Gradient](#toc3_2_5_2_)    
    - [Top-Hat ($\text{TH}$)](#toc3_2_6_)    
      - [Binary Top-Hat](#toc3_2_6_1_)    
      - [Grayscale Top-Hat](#toc3_2_6_2_)    
    - [Black-Hat ($\text{BH}$)](#toc3_2_7_)    
      - [Binary Black-Hat](#toc3_2_7_1_)    
      - [Grayscale Black-Hat](#toc3_2_7_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import skimage as ski
from numpy.typing import NDArray

In [ ]:
# disable automatic figure display (plt.show() required)  
# this ensures consistency with .py scripts and gives full control over when plots appear
plt.ioff()

In [ ]:
# create a reproducible random number generator
rng = np.random.default_rng(seed=42)

# <a id='toc2_'></a>[Prepare Images](#toc0_)


## <a id='toc2_1_'></a>[Binary Images](#toc0_)


In [ ]:
# square with hole (centered)
im_bin_1 = np.zeros((32, 32), dtype=np.uint8)
im_bin_1[8:24, 8:24] = 1
im_bin_1[12:20, 12:20] = 0

In [ ]:
# two close objects
im_bin_2 = np.zeros((32, 32), dtype=np.uint8)
im_bin_2[10:18, 6:12] = 1
im_bin_2[10:18, 14:20] = 1

In [ ]:
# diagonal line structure
im_bin_3 = np.zeros((32, 32), dtype=np.uint8)
for i in range(5, 27):
    im_bin_3[i, i] = 1
    im_bin_3[i, i + 1] = 1
    im_bin_3[i, i - 1] = 1

In [ ]:
# noisy blob
im_bin_4 = np.zeros((32, 32), dtype=np.uint8)
im_bin_4[10:22, 10:22] = 1
im_bin_4 = np.bitwise_or(im_bin_4, rng.integers(0, 2, size=(32, 32)) & (rng.random((32, 32)) < 0.05))
im_bin_4 = im_bin_4.astype(np.uint8)

In [ ]:
# circle with pepper noise
im_bin_5 = np.zeros((32, 32), dtype=np.uint8)
y, x = np.ogrid[-16:16, -16:16]
circle_mask = x**2 + y**2 <= 64  # radius=8
im_bin_5[circle_mask] = 1
pepper_mask = rng.random((32, 32)) < 0.05
im_bin_5[pepper_mask & circle_mask] = 0

In [ ]:
#  irregular shape with protrusions/indentations
im_bin_6 = np.zeros((32, 32), dtype=np.uint8)
im_bin_6[5:25, 5:20] = 1

# protrusions (thin structures)
im_bin_6[3:5, 10:15] = 1
im_bin_6[25:27, 8:12] = 1

# indentations (small holes)
im_bin_6[12:18, 15:18] = 0
im_bin_6[20:22, 7:9] = 0

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=6, figsize=(24, 4), layout="compressed")
axs[0].imshow(im_bin_1, vmin=0, vmax=1, cmap="gray")
axs[0].set_title("im_bin_1 (binary)")
axs[1].imshow(im_bin_2, vmin=0, vmax=1, cmap="gray")
axs[1].set_title("im_bin_2 (binary)")
axs[2].imshow(im_bin_3, vmin=0, vmax=1, cmap="gray")
axs[2].set_title("im_bin_3 (binary)")
axs[3].imshow(im_bin_4, vmin=0, vmax=1, cmap="gray")
axs[3].set_title("im_bin_4 (binary)")
axs[4].imshow(im_bin_5, vmin=0, vmax=1, cmap="gray")
axs[4].set_title("im_bin_5 (binary)")
axs[5].imshow(im_bin_6, vmin=0, vmax=1, cmap="gray")
axs[5].set_title("im_bin_6 (binary)")
plt.show()

## <a id='toc2_2_'></a>[Grayscale Images](#toc0_)


In [ ]:
# gradient ramp with salt-pepper noise
im_gray_1 = np.zeros((32, 32), dtype=np.float32)
im_gray_1 += np.linspace(0, 1, 32).reshape(1, -1)  # Vertical ramp

# add salt & pepper noise
salt_mask = rng.random((32, 32)) < 0.05
pepper_mask = rng.random((32, 32)) < 0.05
im_gray_1[salt_mask] = 1.0
im_gray_1[pepper_mask] = 0.0

In [ ]:
# gaussian hills with valley
y, x = np.mgrid[-16:16, -16:16]
im_gray_2 = np.exp(-(x**2 + y**2) / 50)  # base hill
im_gray_2 += 0.7 * np.exp(-((x - 10) ** 2 + (y + 5) ** 2) / 30)  # right hill
im_gray_2 -= 0.4 * np.exp(-((x + 5) ** 2 + y**2) / 20)  # valley
im_gray_2 = (im_gray_2 - im_gray_2.min()) / (im_gray_2.max() - im_gray_2.min())

In [ ]:
# step edge with texture
im_gray_3 = np.zeros((32, 32), dtype=np.float32)
im_gray_3[:, 16:] = 0.8  # Step edge

# add texture (high-frequency pattern)
texture = rng.random((32, 32)) * 0.3
texture[::2, ::2] += 0.2  # Grid-like pattern
im_gray_3 = np.clip(im_gray_3 + texture, 0, 1)

In [ ]:
# sinusoidal waves
x = np.arange(32)

# vertical waves (column vector)
vertical = 0.3 * np.sin(2 * np.pi * x / 8)

# horizontal waves (row vector)
horizontal = 0.2 * np.cos(2 * np.pi * x / 4)

# add with proper broadcasting
im_gray_4 = 0.5 + vertical[:, np.newaxis] + horizontal[np.newaxis, :]
im_gray_4 = (im_gray_4 - im_gray_4.min()) / (im_gray_4.max() - im_gray_4.min())

In [ ]:
# circular features with noise
y, x = np.ogrid[-16:16, -16:16]
im_gray_5 = np.zeros((32, 32))
im_gray_5[x**2 + y**2 < 64] = 0.9  # bright disk (r=8)
im_gray_5[(x - 8) ** 2 + (y + 8) ** 2 < 25] = 0.2  # dark disk (r=5)

# add gaussian noise
im_gray_5 += rng.normal(0, 0.1, (32, 32))
im_gray_5 = np.clip(im_gray_5, 0, 1)

In [ ]:
# ridge line with gradual slope
im_gray_6 = np.zeros((32, 32))
for r in range(32):
    im_gray_6[r, :] = 0.8 * np.exp(-((r - 16) ** 2) / 20)  # ridge center
im_gray_6 += np.linspace(0, 0.3, 32).reshape(1, -1)  # Left-right gradient

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=6, figsize=(24, 4), layout="compressed")
axs[0].imshow(im_gray_1, vmin=0, vmax=1, cmap="gray")
axs[0].set_title("im_gray_1 (grayscale)")
axs[1].imshow(im_gray_2, vmin=0, vmax=1, cmap="gray")
axs[1].set_title("im_gray_2 (grayscale)")
axs[2].imshow(im_gray_3, vmin=0, vmax=1, cmap="gray")
axs[2].set_title("im_gray_3 (grayscale)")
axs[3].imshow(im_gray_4, vmin=0, vmax=1, cmap="gray")
axs[3].set_title("im_gray_4 (grayscale)")
axs[4].imshow(im_gray_5, vmin=0, vmax=1, cmap="gray")
axs[4].set_title("im_gray_5 (grayscale)")
axs[5].imshow(im_gray_6, vmin=0, vmax=1, cmap="gray")
axs[5].set_title("im_gray_6 (grayscale)")
plt.show()

## <a id='toc2_3_'></a>[RGB Images](#toc0_)


In [ ]:
# RGB gradient with salt-pepper noise
h, w = 32, 32
im_rgb_1 = np.zeros((h, w, 3), dtype=np.float32)

# R: vertical ramp + salt noise
im_rgb_1[..., 0] = np.linspace(0, 1, w).reshape(1, -1)
im_rgb_1[..., 0] += (rng.random((h, w)) < 0.05) * 0.8

# G: horizontal ramp + pepper noise
im_rgb_1[..., 1] = np.linspace(0, 1, h).reshape(-1, 1)
im_rgb_1[..., 1] -= (rng.random((h, w)) < 0.05) * 0.6

# B: diagonal gradient
im_rgb_1[..., 2] = np.linspace(0, 1, h * w).reshape(h, w)

im_rgb_1 = np.clip(im_rgb_1, 0, 1)

In [ ]:
# overlapping color shapes
im_rgb_2 = np.zeros((32, 32, 3), dtype=np.float32)
y, x = np.ogrid[-16:16, -16:16]

# R: red square
im_rgb_2[8:24, 4:20, 0] = 0.9

# G: green circle (overlaps square)
circle_mask = x**2 + y**2 < 64
im_rgb_2[..., 1][circle_mask] = 0.7

# B: blue triangle (overlaps both)
for r in range(32):
    im_rgb_2[r, max(0, 12 - r) : min(32, 20 + r), 2] = 0.8

In [ ]:
# multi-channel wave patterns
x = np.arange(32)
im_rgb_3 = np.zeros((32, 32, 3))

# R: vertical waves
im_rgb_3[..., 0] = 0.4 + 0.3 * np.sin(2 * np.pi * x / 8)[:, None]

# G: horizontal waves
im_rgb_3[..., 1] = 0.5 + 0.2 * np.cos(2 * np.pi * x / 6)[None, :]

# B: diagonal waves
im_rgb_3[..., 2] = 0.3 + 0.4 * np.sin(2 * np.pi * (x[:, None] + x[None, :]) / 10)

im_rgb_3 = np.clip(im_rgb_3, 0, 1)

In [ ]:
# noisy color blobs with holes
im_rgb_4 = np.zeros((32, 32, 3))
centers = [(10, 10), (10, 22), (22, 16)]

# create blobs per channel
for c in range(3):
    yc, xc = centers[c]
    for y in range(32):
        for x in range(32):
            dist = np.sqrt((x - xc) ** 2 + (y - yc) ** 2)
            im_rgb_4[y, x, c] = max(0, 0.8 - 0.05 * dist)

    # add channel-specific holes
    hole_size = 5 + c * 2
    im_rgb_4[yc - 3 : yc + 3, xc - 3 : xc + 3, c] = 0

    # add different noise per channel
    noise = rng.random((32, 32)) * 0.3
    if c == 0:  # salt only on red
        noise[noise > 0.8] = 0.9
    im_rgb_4[..., c] = np.clip(im_rgb_4[..., c] + noise, 0, 1)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 4), layout="compressed")
axs[0].imshow(im_rgb_1, vmin=0, vmax=1)
axs[0].set_title("im_rgb_1 (rgb)")
axs[1].imshow(im_rgb_2, vmin=0, vmax=1)
axs[1].set_title("im_rgb_2 (rgb)")
axs[2].imshow(im_rgb_3, vmin=0, vmax=1)
axs[2].set_title("im_rgb_3 (rgb)")
axs[3].imshow(im_rgb_4, vmin=0, vmax=1)
axs[3].set_title("im_rgb_4 (rgb)")
plt.show()

WE NEED vector morphology to consider
WHAT IS anisotropic MORPHOLOGY

# <a id='toc3_'></a>[Morphological Processing](#toc0_)

- Morphological image processing refers to a set of operations that process images based on shapes.
- These techniques are typically applied to binary images and rely on structuring elements to probe and transform input images.

📝 **Docs**:
- Morphological Transformations: [docs.opencv.org/master/d4/d76/tutorial_js_morphological_ops.html](https://docs.opencv.org/master/d4/d76/tutorial_js_morphological_ops.html)
- Image Filtering: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb)
- `cv2.erode`: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb)
- `cv2.dilate`: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga4ff0f3318642c4f469d0e11f242f3b6c)
- `cv2.morphologyEx`: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga67493776e3ad1a3df63883829375201f](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga67493776e3ad1a3df63883829375201f)
- `cv2.getStructuringElement`: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac342a1bb6eabf6f55c803b09268e36dc](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#gac342a1bb6eabf6f55c803b09268e36dc)
- `cv2.MorphTypes`: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga7be549266bad7b2e6a04db49827f9f32](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga7be549266bad7b2e6a04db49827f9f32)
- `skimage.morphology`: [scikit-image.org/docs/stable/api/skimage.morphology.html](https://scikit-image.org/docs/stable/api/skimage.morphology.html)
- `scipy.ndimage`: [docs.scipy.org/doc/scipy/reference/ndimage.html#morphology](https://docs.scipy.org/doc/scipy/reference/ndimage.html#morphology)
- Multidimensional Image Processing: [docs.scipy.org/doc/scipy/tutorial/ndimage.html#morphology](https://docs.scipy.org/doc/scipy/tutorial/ndimage.html#morphology)

## <a id='toc3_1_'></a>[Structuring Elements](#toc0_)

**Common Structuring Elements:**

$$
\begin{array}{ccccccc}
\begin{bmatrix}1 & 1 & 1 \\ 1 & 1 & 1 \\ 1 & 1 & 1\end{bmatrix}, &
\begin{bmatrix}0 & 1 & 0 \\ 1 & 1 & 1 \\ 0 & 1 & 0\end{bmatrix}, &
\begin{bmatrix}0 & 0 & 0 \\ 1 & 1 & 1 \\ 0 & 0 & 0\end{bmatrix}, &
\begin{bmatrix}0 & 0 & 1 \\ 0 & 1 & 0 \\ 1 & 0 & 0\end{bmatrix}, &
\begin{bmatrix}0 & 0 & 1 & 0 & 0 \\ 0 & 1 & 1 & 1 & 0 \\ 1 & 1 & 1 & 1 & 1 \\ 0 & 1 & 1 & 1 & 0 \\ 0 & 0 & 1 & 0 & 0\end{bmatrix}, &
\begin{bmatrix}0 & 0 & 1 & 0 & 0 \\ 0 & 1 & 1 & 1 & 0 \\ 1 & 1 & 1 & 1 & 1 \\ 0 & 1 & 1 & 1 & 0 \\ 0 & 0 & 1 & 0 & 0\end{bmatrix}, &
\begin{bmatrix}0 & 1 & 1 & 1 & 0 \\ 1 & 1 & 1 & 1 & 1 \\ 1 & 1 & 1 & 1 & 1 \\ 1 & 1 & 1 & 1 & 1 \\ 0 & 1 & 1 & 1 & 0\end{bmatrix} \\
\text{(a)} & \text{(b)} & \text{(c)} & \text{(d)} & \text{(e)} & \text{(f)} & \text{(g)}
\end{array}
$$

- **(a) Square (Box)**: General-purpose operations, such as smoothing, noise removal, or filling small holes.
- **(b) Cross**: Operations that require preserving connectivity or working with thin structures.
- **(c) Line 1**:  Operations that involve directional features or linear structures.
- **(d) Line 2 (Diagonal)**: Operations that involve directional features or linear structures.
- **(e) Diamond**: Operations that require isotropic behavior but with a softer shape than a square.
- **(f) Disk (radius = 2)**: Operations that require circular symmetry or rounded shapes.
- **(g) Octagon (radius = 2)**: Operations that require a balance between square and disk shapes.

### <a id='toc3_1_1_'></a>[Square](#toc0_)


In [ ]:
se_square2 = np.ones((2, 2), dtype=np.uint8)
se_square3 = np.ones((3, 3), dtype=np.uint8)
se_square5 = np.ones((5, 5), dtype=np.uint8)

# log
print(f"se_square2:\n{se_square2}\n")
print(f"se_square3:\n{se_square3}\n")
print(f"se_square5:\n{se_square5}")

In [ ]:
se_square3_ski = ski.morphology.footprint_rectangle((3, 3))
se_square5_ski = ski.morphology.footprint_rectangle((5, 5))

# log
print(f"se_square3_ski:\n{se_square3_ski}\n")
print(f"se_square5_ski:\n{se_square5_ski}")

### <a id='toc3_1_2_'></a>[Disk (Circular)](#toc0_)


In [ ]:
se_disk1 = ski.morphology.disk(1)
se_disk3 = ski.morphology.disk(3)
se_disk4 = ski.morphology.disk(4)
se_disk5 = ski.morphology.disk(5)

# log
print(f"se_disk1:\n{se_disk1}\n")
print(f"se_disk3:\n{se_disk3}\n")
print(f"se_disk4:\n{se_disk4}\n")
print(f"se_disk5:\n{se_disk5}")

### <a id='toc3_1_3_'></a>[Diamond](#toc0_)


In [ ]:
se_diamond1 = ski.morphology.diamond(1)
se_diamond2 = ski.morphology.diamond(2)

# log
print(f"se_diamond1:\n{se_diamond1}\n")
print(f"se_diamond2:\n{se_diamond2}")

### <a id='toc3_1_4_'></a>[Octagon](#toc0_)


In [ ]:
se_octagon3_3 = ski.morphology.octagon(3, 3)
se_octagon4_2 = ski.morphology.octagon(4, 2)

# log
print(f"se_octagon3_3:\n{se_octagon3_3}\n")
print(f"se_octagon4_2:\n{se_octagon4_2}")

### <a id='toc3_1_5_'></a>[Line](#toc0_)


In [ ]:
se_line_h = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 1))
se_line_v = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 7))

# log
print(f"se_line_h:\n{se_line_h}\n")
print(f"se_line_v:\n{se_line_v}")

In [ ]:
se_line_45 = np.zeros((5, 5), dtype=np.uint8)
cv2.line(se_line_45, (0, 4), (4, 0), 1, thickness=1)

se_line_135 = np.zeros((7, 7), dtype=np.uint8)
cv2.line(se_line_135, (0, 0), (6, 6), 1, thickness=1)

# log
print(f"se_line_45:\n{se_line_45}\n")
print(f"se_line_135:\n{se_line_135}")

### <a id='toc3_1_6_'></a>[Cross (Plus)](#toc0_)


In [ ]:
se_cross3 = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], dtype=np.uint8)

# log
print(f"se_cross3:\n{se_cross3}")

In [ ]:
se_cross3_ski = ski.morphology.diamond(1)

# log
print(f"se_cross3_ski:\n{se_cross3_ski}")

### <a id='toc3_1_7_'></a>[Rectangle](#toc0_)


In [ ]:
se_rect_3x7 = ski.morphology.footprint_rectangle((3, 7))
se_rect_7x3 = ski.morphology.footprint_rectangle((7, 3))

# log
print(f"se_rect_3x7:\n{se_rect_3x7}\n")
print(f"se_rect_7x3:\n{se_rect_7x3}")

## <a id='toc3_2_'></a>[Morphological Operators](#toc0_)


### <a id='toc3_2_1_'></a>[Erosion ($\ominus$)](#toc0_)

**Erosion** is a fundamental morphological operation that shrinks bright (foreground) regions in an image.


In [ ]:
def erosion(image: NDArray, se: NDArray) -> NDArray:

    H, W = image.shape
    h, w = se.shape
    pad_h = h // 2
    pad_w = w // 2

    # pad image with zeros on all sides
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode="constant", constant_values=0)

    # precompute coordinates of SE’s 1‐entries
    se_coords = np.argwhere(se == 1)
    offsets_i = se_coords[:, 0]
    offsets_j = se_coords[:, 1]

    eroded = np.empty((H, W), dtype=image.dtype)

    # for each pixel in the original image, compute the min over the neighborhood
    for i in range(H):
        for j in range(W):
            # window top-left in padded is (i, j), bottom-right (i+h−1, j+w−1)
            # but we only sample positions where se == 1:
            vals = padded[i + offsets_i, j + offsets_j]
            eroded[i, j] = vals.min()

    return eroded

#### <a id='toc3_2_1_1_'></a>[Binary Erosion](#toc0_)
- Shrinks foreground objects.
- Removes small white noise and disconnects connected components.

$$
I \ominus S = \{ x \mid S_x \subseteq I \}
$$

Where:  
- $I$: Input binary image  
- $S$: Structuring element  
- $S_x$: Structuring element translated to position $x$


##### <a id='toc3_2_1_1_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
erode_bin_1 = cv2.erode(im_bin_1, se_line_45)
erode_bin_2 = cv2.erode(im_bin_3, se_line_135)
erode_bin_3 = cv2.erode(im_bin_5, se_square3)
erode_bin_4 = cv2.erode(im_bin_6, se_diamond1)

imgs = [im_bin_1, im_bin_3, im_bin_5, im_bin_6]
erodes = [erode_bin_1, erode_bin_2, erode_bin_3, erode_bin_4]
titles = ["im_bin_1 & se_line_45", "im_bin_3 & se_line_135", "im_bin_5 & se_square3", "im_bin_6 & se_diamond1"]

# plot
fig, axs = plt.subplots(3, len(erodes), figsize=(16, 12), layout="compressed")
for col, (img, erode, title) in enumerate(zip(imgs, erodes, titles)):
    deleted_idx = (img == 1) & (erode == 0)
    erode_rgb = np.stack([erode * 255] * 3, axis=-1)
    erode_rgb[deleted_idx] = [255, 0, 0]
    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(erode_rgb)
    axs[1, col].set_title(f"Erosion: {title}")
    axs[2, col].imshow(erode, cmap="gray")
    axs[2, col].set_title(f"Final image")
plt.show()

##### <a id='toc3_2_1_1_2_'></a>[Using Scikit-Image](#toc0_)


In [ ]:
erode_bin_1 = ski.morphology.erosion(im_bin_1, footprint=se_line_45)
erode_bin_2 = ski.morphology.erosion(im_bin_3, footprint=se_line_135)
erode_bin_3 = ski.morphology.erosion(im_bin_5, footprint=se_square3)
erode_bin_4 = ski.morphology.erosion(im_bin_6, footprint=se_diamond1)

imgs = [im_bin_1, im_bin_3, im_bin_5, im_bin_6]
erodes = [erode_bin_1, erode_bin_2, erode_bin_3, erode_bin_4]
titles = ["im_bin_1 & se_line_45", "im_bin_3 & se_line_135", "im_bin_5 & se_square3", "im_bin_6 & se_diamond1"]

# plot
fig, axs = plt.subplots(3, len(erodes), figsize=(16, 12), layout="compressed")
for col, (img, erode, title) in enumerate(zip(imgs, erodes, titles)):
    deleted_idx = (img == 1) & (erode == 0)
    erode_rgb = np.stack([erode * 255] * 3, axis=-1)
    erode_rgb[deleted_idx] = [255, 0, 0]
    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(erode_rgb)
    axs[1, col].set_title(f"Erosion: {title}")
    axs[2, col].imshow(erode, cmap="gray")
    axs[2, col].set_title(f"Final image")
plt.show()

##### <a id='toc3_2_1_1_3_'></a>[Using SciPy](#toc0_)

In [ ]:
erode_bin_1 = sp.ndimage.binary_erosion(im_bin_1, structure=se_line_45)
erode_bin_2 = sp.ndimage.binary_erosion(im_bin_3, structure=se_line_135)
erode_bin_3 = sp.ndimage.binary_erosion(im_bin_5, structure=se_square3)
erode_bin_4 = sp.ndimage.binary_erosion(im_bin_6, structure=se_diamond1)

imgs = [im_bin_1, im_bin_3, im_bin_5, im_bin_6]
erodes = [erode_bin_1, erode_bin_2, erode_bin_3, erode_bin_4]
titles = ["im_bin_1 & se_line_45", "im_bin_3 & se_line_135", "im_bin_5 & se_square3", "im_bin_6 & se_diamond1"]

# plot
fig, axs = plt.subplots(3, len(erodes), figsize=(16, 12), layout="compressed")
for col, (img, erode, title) in enumerate(zip(imgs, erodes, titles)):
    deleted_idx = (img == 1) & (erode == 0)
    erode_rgb = np.stack([erode * 255] * 3, axis=-1)
    erode_rgb[deleted_idx] = [255, 0, 0]
    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(erode_rgb)
    axs[1, col].set_title(f"Erosion: {title}")
    axs[2, col].imshow(erode, cmap="gray")
    axs[2, col].set_title(f"Final image")
plt.show()

#### <a id='toc3_2_1_2_'></a>[Grayscale Erosion](#toc0_)
- Darkens bright regions by taking the local minimum.
- Suppresses small bright spots and thins object boundaries.

$$
(I \ominus S)(x) = \min_{s \in S} \left[ I(x + s) - S(s) \right]
$$

Where:  
- $I$: Input grayscale image  
- $S$: Structuring element (can be flat or non-flat)  
- $s$: Offset inside the structuring element

> **Note:** For RGB images, erosion is typically applied **channel-wise** by treating each color channel (R, G, B) as a separate grayscale image.


In [ ]:
erode_gray_1 = cv2.erode(im_gray_1, se_disk1)
erode_gray_2 = cv2.erode(im_gray_2, se_disk3)
erode_gray_3 = cv2.erode(im_gray_3, se_square3)
erode_gray_4 = cv2.erode(im_gray_4, se_square5)

imgs_gray = [im_gray_1, im_gray_2, im_gray_3, im_gray_4]
erodes_gray = [erode_gray_1, erode_gray_2, erode_gray_3, erode_gray_4]
titles = ["im_gray_1 & se_disk1", "im_gray_2 & se_disk3", "im_gray_3 & se_square3", "im_gray_4 & se_square5"]

fig, axs = plt.subplots(3, len(erodes_gray), figsize=(16, 12), layout="compressed")
for col, (img, erode, title) in enumerate(zip(imgs_gray, erodes_gray, titles)):
    diff_img = np.abs(img.astype(np.float32) - erode)

    axs[0, col].imshow(img, cmap="gray", vmin=0, vmax=1)
    axs[0, col].set_title("Original Grayscale")
    axs[1, col].imshow(erode, cmap="gray", vmin=0, vmax=1)
    axs[1, col].set_title(f"Erosion: {title}")
    axs[2, col].imshow(diff_img, cmap="hot", vmin=0, vmax=1)
    axs[2, col].set_title("Absolute difference")
plt.show()

### <a id='toc3_2_2_'></a>[Dilation ($\oplus$)](#toc0_)

**Dilation** is a fundamental morphological operation that expands bright (foreground) regions in an image.


In [ ]:
def dilation(image: NDArray, se: NDArray) -> NDArray:

    H, W = image.shape
    h, w = se.shape
    pad_h = h // 2
    pad_w = w // 2

    # pad image with zeros on all sides
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode="constant", constant_values=0)

    # precompute coordinates of SE’s 1‐entries
    se_coords = np.argwhere(se == 1)
    offsets_i = se_coords[:, 0]
    offsets_j = se_coords[:, 1]

    dilated = np.empty((H, W), dtype=image.dtype)

    # for each pixel in the original image, compute the max over the neighborhood
    for i in range(H):
        for j in range(W):
            vals = padded[i + offsets_i, j + offsets_j]
            dilated[i, j] = vals.max()

    return dilated

#### <a id='toc3_2_2_1_'></a>[Binary Dilation](#toc0_)
- Expands foreground objects.
- Fills small holes and connects nearby objects.

$$
I \oplus S = \{ x \mid (Ŝ)_x \cap I \neq \emptyset \}
$$

Where:  
- $I$: Input binary image  
- $S$: Structuring element  
- $Ŝ$: Reflection of structuring element $S$  
- $(Ŝ)_x$: Reflected structuring element translated to position $x$


In [ ]:
dilate_bin_1 = cv2.dilate(im_bin_2, se_square3)
dilate_bin_2 = cv2.dilate(im_bin_3, se_line_45)
dilate_bin_3 = cv2.dilate(im_bin_5, se_disk1)
dilate_bin_4 = cv2.dilate(im_bin_6, se_diamond1)

imgs = [im_bin_2, im_bin_3, im_bin_5, im_bin_6]
dilates = [dilate_bin_1, dilate_bin_2, dilate_bin_3, dilate_bin_4]
titles = ["im_bin_2 & se_square3", "im_bin_3 & se_line_45", "im_bin_5 & se_disk1", "im_bin_6 & se_diamond1"]

fig, axs = plt.subplots(3, len(dilates), figsize=(16, 12), layout="compressed")
for col, (img, dilate, title) in enumerate(zip(imgs, dilates, titles)):
    added_idx = (img == 0) & (dilate == 1)
    dilate_rgb = np.stack([dilate * 255] * 3, axis=-1)
    dilate_rgb[added_idx] = [0, 255, 0]

    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(dilate_rgb)
    axs[1, col].set_title(f"Dilation: {title}")
    axs[2, col].imshow(dilate, cmap="gray")
    axs[2, col].set_title("Final image")
plt.show()

#### <a id='toc3_2_2_2_'></a>[Grayscale Dilation](#toc0_)
- Brightens regions by taking the local maximum.
- Fills small dark spots and thickens object boundaries.

$$
(I \oplus S)(x) = \max_{s \in S} \left[ I(x - s) + S(s) \right]
$$

Where:  
- $I$: Input grayscale image  
- $S$: Structuring element (can be flat or non-flat)  
- $s$: Offset inside the structuring element

> **Note:** For RGB images, dilation is typically applied **channel-wise** by treating each color channel (R, G, B) as a separate grayscale image.


In [ ]:
dilate_gray_1 = cv2.dilate(im_gray_1, se_disk1)
dilate_gray_2 = cv2.dilate(im_gray_2, se_disk3)
dilate_gray_3 = cv2.dilate(im_gray_3, se_square3)
dilate_gray_4 = cv2.dilate(im_gray_4, se_square5)

imgs_gray = [im_gray_1, im_gray_2, im_gray_3, im_gray_4]
dilates_gray = [dilate_gray_1, dilate_gray_2, dilate_gray_3, dilate_gray_4]
titles = ["im_gray_1 & se_disk1", "im_gray_2 & se_disk3", "im_gray_3 & se_square3", "im_gray_4 & se_square5"]

fig, axs = plt.subplots(3, len(dilates_gray), figsize=(16, 12), layout="compressed")
for col, (img, dilate, title) in enumerate(zip(imgs_gray, dilates_gray, titles)):
    diff_img = np.abs(dilate - img.astype(np.float32))

    axs[0, col].imshow(img, cmap="gray", vmin=0, vmax=1)
    axs[0, col].set_title("Original Grayscale")
    axs[1, col].imshow(dilate, cmap="gray", vmin=0, vmax=1)
    axs[1, col].set_title(f"Dilation: {title}")
    axs[2, col].imshow(diff_img, cmap="hot", vmin=0, vmax=1)
    axs[2, col].set_title("Absolute difference")
plt.show()

### <a id='toc3_2_3_'></a>[Opening ($\circ$)](#toc0_)

**Opening** is a morphological operation that smooths object contours, removes small objects, and separates objects that are close together by performing erosion followed by dilation.


In [ ]:
def opening(image: NDArray, se: NDArray) -> NDArray:
    eroded = erosion(image, se)
    opened = dilation(eroded, se)
    return opened

#### <a id='toc3_2_3_1_'></a>[Binary Opening](#toc0_)
- Removes small foreground objects (noise).
- Smooths object boundaries without significantly changing their area.

$$
I \circ S = (I \ominus S) \oplus S
$$

Where:  
- $I$: Input binary image  
- $S$: Structuring element


In [ ]:
open_bin_1 = cv2.morphologyEx(im_bin_4, cv2.MORPH_OPEN, se_square3)
open_bin_2 = cv2.morphologyEx(im_bin_6, cv2.MORPH_OPEN, se_diamond1)
open_bin_3 = cv2.morphologyEx(im_bin_3, cv2.MORPH_OPEN, se_line_45)
open_bin_4 = cv2.morphologyEx(im_bin_2, cv2.MORPH_OPEN, se_disk1)

imgs = [im_bin_4, im_bin_6, im_bin_3, im_bin_2]
opens = [open_bin_1, open_bin_2, open_bin_3, open_bin_4]
titles = ["im_bin_4 & se_square3", "im_bin_6 & se_diamond1", "im_bin_3 & se_line_45", "im_bin_2 & se_disk1"]

fig, axs = plt.subplots(3, len(opens), figsize=(16, 12), layout="compressed")
for col, (img, open_img, title) in enumerate(zip(imgs, opens, titles)):
    deleted_idx = (img == 1) & (open_img == 0)
    open_rgb = np.stack([open_img * 255] * 3, axis=-1)
    open_rgb[deleted_idx] = [255, 0, 0]

    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(open_rgb)
    axs[1, col].set_title(f"Opening: {title}")
    axs[2, col].imshow(open_img, cmap="gray")
    axs[2, col].set_title("Final image")
plt.show()

#### <a id='toc3_2_3_2_'></a>[Grayscale Opening](#toc0_)
- Removes small bright spots and smooths contours.
- Preserves overall shape but eliminates small details.

$$
(I \circ S)(x) = \left( (I \ominus S) \oplus S \right)(x)
$$

Where:  
- $I$: Input grayscale image  
- $S$: Structuring element

> **Note:** For RGB images, opening is typically applied **channel-wise** by treating each color channel (R, G, B) as a separate grayscale image.


In [ ]:
open_gray_1 = cv2.morphologyEx(im_gray_1, cv2.MORPH_OPEN, se_disk1)
open_gray_2 = cv2.morphologyEx(im_gray_3, cv2.MORPH_OPEN, se_square3)
open_gray_3 = cv2.morphologyEx(im_gray_4, cv2.MORPH_OPEN, se_square5)
open_gray_4 = cv2.morphologyEx(im_gray_5, cv2.MORPH_OPEN, se_disk1)

imgs_gray = [im_gray_1, im_gray_3, im_gray_4, im_gray_5]
opens_gray = [open_gray_1, open_gray_2, open_gray_3, open_gray_4]
titles = ["im_gray_1 & se_disk1", "im_gray_3 & se_square3", "im_gray_4 & se_square5", "im_gray_5 & se_disk1"]

fig, axs = plt.subplots(3, len(opens_gray), figsize=(16, 12), layout="compressed")
for col, (img, opened, title) in enumerate(zip(imgs_gray, opens_gray, titles)):
    diff_img = np.abs(img.astype(np.float32) - opened)

    axs[0, col].imshow(img, cmap="gray", vmin=0, vmax=1)
    axs[0, col].set_title("Original Grayscale")
    axs[1, col].imshow(opened, cmap="gray", vmin=0, vmax=1)
    axs[1, col].set_title(f"Opening: {title}")
    axs[2, col].imshow(diff_img, cmap="hot", vmin=0, vmax=1)
    axs[2, col].set_title("Absolute difference")
plt.show()

### <a id='toc3_2_4_'></a>[Closing ($\bullet$)](#toc0_)

**Closing** is a morphological operation that smooths object contours, fills small holes, and connects nearby objects by performing dilation followed by erosion.


In [ ]:
def closing(image: NDArray, se: NDArray) -> NDArray:
    dilated = dilation(image, se)
    closed = erosion(dilated, se)
    return closed

#### <a id='toc3_2_4_1_'></a>[Binary Closing](#toc0_)
- Fills small holes and gaps inside foreground objects.
- Connects nearby objects without significantly changing their area.

$$
I \bullet S = (I \oplus S) \ominus S
$$

Where:  
- $I$: Input binary image  
- $S$: Structuring element


In [ ]:
close_bin_1 = cv2.morphologyEx(im_bin_5, cv2.MORPH_CLOSE, se_disk1)
close_bin_2 = cv2.morphologyEx(im_bin_2, cv2.MORPH_CLOSE, se_square3)
close_bin_3 = cv2.morphologyEx(im_bin_1, cv2.MORPH_CLOSE, se_disk3)
close_bin_4 = cv2.morphologyEx(im_bin_6, cv2.MORPH_CLOSE, se_diamond1)

imgs = [im_bin_5, im_bin_2, im_bin_1, im_bin_6]
closes = [close_bin_1, close_bin_2, close_bin_3, close_bin_4]
titles = ["im_bin_5 & se_line_45", "im_bin_2 & se_line_135", "im_bin_1 & se_square3", "im_bin_6 & se_diamond1"]

fig, axs = plt.subplots(3, len(closes), figsize=(16, 12), layout="compressed")
for col, (img, close_img, title) in enumerate(zip(imgs, closes, titles)):
    added_idx = (img == 0) & (close_img == 1)
    close_rgb = np.stack([close_img * 255] * 3, axis=-1)
    close_rgb[added_idx] = [0, 255, 0]

    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(close_rgb)
    axs[1, col].set_title(f"Closing: {title}")
    axs[2, col].imshow(close_img, cmap="gray")
    axs[2, col].set_title("Final image")
plt.show()

#### <a id='toc3_2_4_2_'></a>[Grayscale Closing](#toc0_)
- Fills small dark spots and smooths contours.
- Preserves overall shape but fills small gaps.

$$
(I \bullet S)(x) = \left( (I \oplus S) \ominus S \right)(x)
$$

Where:  
- $I$: Input grayscale image  
- $S$: Structuring element

> **Note:** For RGB images, closing is typically applied **channel-wise** by treating each color channel (R, G, B) as a separate grayscale image.


In [ ]:
close_gray_1 = cv2.morphologyEx(im_gray_1, cv2.MORPH_CLOSE, se_disk1)
close_gray_2 = cv2.morphologyEx(im_gray_2, cv2.MORPH_CLOSE, se_disk3)
close_gray_3 = cv2.morphologyEx(im_gray_3, cv2.MORPH_CLOSE, se_square3)
close_gray_4 = cv2.morphologyEx(im_gray_4, cv2.MORPH_CLOSE, se_square5)

imgs_gray = [im_gray_1, im_gray_2, im_gray_3, im_gray_4]
closes_gray = [close_gray_1, close_gray_2, close_gray_3, close_gray_4]
titles = ["im_gray_1 & se_disk1", "im_gray_2 & se_disk3", "im_gray_3 & se_square3", "im_gray_4 & se_square5"]

fig, axs = plt.subplots(3, len(closes_gray), figsize=(16, 12), layout="compressed")
for col, (img, closed, title) in enumerate(zip(imgs_gray, closes_gray, titles)):
    diff_img = np.abs(img.astype(np.float32) - closed)

    axs[0, col].imshow(img, cmap="gray", vmin=0, vmax=1)
    axs[0, col].set_title("Original Grayscale")
    axs[1, col].imshow(closed, cmap="gray", vmin=0, vmax=1)
    axs[1, col].set_title(f"Closing: {title}")
    axs[2, col].imshow(diff_img, cmap="hot", vmin=0, vmax=1)
    axs[2, col].set_title("Absolute difference")
plt.show()

### <a id='toc3_2_5_'></a>[Gradient ($\nabla$)](#toc0_)

**Morphological Gradient** is an operator that extracts object boundaries by computing the difference between dilation and erosion of the image.


In [ ]:
def gradient(image: NDArray, se: NDArray) -> NDArray:
    dil = dilation(image, se)
    ero = erosion(image, se)
    grad = dil - ero
    return grad

#### <a id='toc3_2_5_1_'></a>[Binary Gradient](#toc0_)
- Highlights the edges of objects in a binary image.
- Useful for boundary detection and shape analysis.

$$
\nabla I = (I \oplus S) - (I \ominus S)
$$

Where:  
- $I$: Input binary image  
- $S$: Structuring element


In [ ]:
grad_bin_1 = cv2.morphologyEx(im_bin_1, cv2.MORPH_GRADIENT, se_square3)
grad_bin_2 = cv2.morphologyEx(im_bin_3, cv2.MORPH_GRADIENT, se_disk1)
grad_bin_3 = cv2.morphologyEx(im_bin_5, cv2.MORPH_GRADIENT, se_square2)
grad_bin_4 = cv2.morphologyEx(im_bin_6, cv2.MORPH_GRADIENT, se_square2)

imgs = [im_bin_1, im_bin_3, im_bin_5, im_bin_6]
grads = [grad_bin_1, grad_bin_2, grad_bin_3, grad_bin_4]
titles = ["im_bin_1 & se_square3", "im_bin_3 & se_disk1", "im_bin_5 & se_square2", "im_bin_6 & se_diamond1"]

fig, axs = plt.subplots(2, len(grads), figsize=(16, 8), layout="compressed")
for col, (img, grad_img, title) in enumerate(zip(imgs, grads, titles)):
    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(grad_img, cmap="gray")
    axs[1, col].set_title("Edges highlighted")
plt.show()

#### <a id='toc3_2_5_2_'></a>[Grayscale Gradient](#toc0_)
- Emphasizes transitions or edges in grayscale images.
- Helps in edge detection while preserving shape.

$$
(\nabla I)(x) = \left( (I \oplus S) - (I \ominus S) \right)(x)
$$

Where:  
- $I$: Input grayscale image  
- $S$: Structuring element

> **Note:** For RGB images, the gradient is usually applied **channel-wise** by processing each channel separately (R, G, B).


In [ ]:
gradient_gray_1 = cv2.morphologyEx(im_gray_1, cv2.MORPH_GRADIENT, se_square3)
gradient_gray_2 = cv2.morphologyEx(im_gray_2, cv2.MORPH_GRADIENT, se_disk3)
gradient_gray_3 = cv2.morphologyEx(im_gray_3, cv2.MORPH_GRADIENT, se_square3)
gradient_gray_4 = cv2.morphologyEx(im_gray_4, cv2.MORPH_GRADIENT, se_square5)

imgs_gray = [im_gray_1, im_gray_2, im_gray_3, im_gray_4]
gradients_gray = [gradient_gray_1, gradient_gray_2, gradient_gray_3, gradient_gray_4]
titles = ["im_gray_1 & se_square3", "im_gray_2 & se_disk3", "im_gray_3 & se_square3", "im_gray_4 & se_square5"]

fig, axs = plt.subplots(3, len(gradients_gray), figsize=(16, 12), layout="compressed")
for col, (img, gradient, title) in enumerate(zip(imgs_gray, gradients_gray, titles)):
    diff_img = np.abs(img.astype(np.float32) - gradient)

    axs[0, col].imshow(img, cmap="gray", vmin=0, vmax=1)
    axs[0, col].set_title("Original Grayscale")
    axs[1, col].imshow(gradient, cmap="gray", vmin=0, vmax=1)
    axs[1, col].set_title(f"Gradient: {title}")
    axs[2, col].imshow(diff_img, cmap="hot", vmin=0, vmax=1)
    axs[2, col].set_title("Absolute difference")
plt.show()

### <a id='toc3_2_6_'></a>[Top-Hat ($\text{TH}$)](#toc0_)

**Top-Hat** transformation extracts small bright features that are smaller than the structuring element by subtracting the opened image from the original.


In [ ]:
def top_hat(image: NDArray, se: NDArray) -> NDArray:
    opened = opening(image, se)
    th = image - opened
    return th

#### <a id='toc3_2_6_1_'></a>[Binary Top-Hat](#toc0_)
- Extracts small bright objects or details.
- Useful for enhancing bright spots on a dark background.

$$
\text{TH}(I) = I - (I \circ S)
$$

Where:  
- $I$: Input binary image  
- $S$: Structuring element


In [ ]:
tophat_bin_1 = cv2.morphologyEx(im_bin_4, cv2.MORPH_TOPHAT, se_square3)
tophat_bin_2 = cv2.morphologyEx(im_bin_6, cv2.MORPH_TOPHAT, se_diamond1)
tophat_bin_3 = cv2.morphologyEx(im_bin_3, cv2.MORPH_TOPHAT, se_square3)
tophat_bin_4 = cv2.morphologyEx(im_bin_2, cv2.MORPH_TOPHAT, se_disk4)

imgs = [im_bin_4, im_bin_6, im_bin_3, im_bin_2]
tophats = [tophat_bin_1, tophat_bin_2, tophat_bin_3, tophat_bin_4]
titles = ["im_bin_4 & se_square3", "im_bin_6 & se_diamond1", "im_bin_3 & se_square3", "im_bin_2 & se_disk4"]

fig, axs = plt.subplots(2, len(tophats), figsize=(16, 8), layout="compressed")
for col, (img, tophat_img, title) in enumerate(zip(imgs, tophats, titles)):
    highlight_idx = tophat_img > 0
    tophat_rgb = np.zeros((*tophat_img.shape, 3), dtype=np.uint8)
    tophat_rgb[highlight_idx] = [255, 0, 0]

    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(tophat_rgb)
    axs[1, col].set_title(f"Top-Hat: {title}")
plt.show()

#### <a id='toc3_2_6_2_'></a>[Grayscale Top-Hat](#toc0_)
- Highlights small bright regions in grayscale images.
- Helps in background correction and feature extraction.

$$
(\text{TH}(I))(x) = I(x) - \left( (I \circ S) \right)(x)
$$

Where:  
- $I$: Input grayscale image  
- $S$: Structuring element

> **Note:** For RGB images, top-hat is typically applied **channel-wise** by processing each color channel separately (R, G, B).


In [ ]:
# Apply grayscale Top-Hat (original - opening)
tophat_gray_1 = cv2.morphologyEx(im_gray_1, cv2.MORPH_TOPHAT, se_disk1)
tophat_gray_2 = cv2.morphologyEx(im_gray_3, cv2.MORPH_TOPHAT, se_square3)
tophat_gray_3 = cv2.morphologyEx(im_gray_5, cv2.MORPH_TOPHAT, se_disk5)
tophat_gray_4 = cv2.morphologyEx(im_gray_2, cv2.MORPH_TOPHAT, se_disk3)

imgs_gray = [im_gray_1, im_gray_3, im_gray_5, im_gray_2]
tophats_gray = [tophat_gray_1, tophat_gray_2, tophat_gray_3, tophat_gray_4]
titles = ["im_gray_1 & se_disk1", "im_gray_3 & se_square3", "im_gray_5 & se_disk5", "im_gray_2 & se_disk3"]

fig, axs = plt.subplots(3, len(tophats_gray), figsize=(16, 12), layout="compressed")
for col, (img, tophat, title) in enumerate(zip(imgs_gray, tophats_gray, titles)):
    diff_img = np.abs(img.astype(np.float32) - tophat)

    axs[0, col].imshow(img, cmap="gray", vmin=0, vmax=1)
    axs[0, col].set_title("Original Grayscale")
    axs[1, col].imshow(tophat, cmap="gray", vmin=0, vmax=1)
    axs[1, col].set_title(f"Top-Hat: {title}")
    axs[2, col].imshow(diff_img, cmap="hot", vmin=0, vmax=1)
    axs[2, col].set_title("Absolute difference")
plt.show()

### <a id='toc3_2_7_'></a>[Black-Hat ($\text{BH}$)](#toc0_)

**Black-Hat** transformation extracts small dark features that are smaller than the structuring element by subtracting the original image from its closing.


In [ ]:
def black_hat(image: NDArray, se: NDArray) -> NDArray:
    closed = closing(image, se)
    bh = closed - image
    return bh

#### <a id='toc3_2_7_1_'></a>[Binary Black-Hat](#toc0_)
- Extracts small dark spots or holes.
- Useful for detecting dark features on a bright background.

$$
\text{BH}(I) = (I \bullet S) - I
$$

Where:  
- $I$: Input binary image  
- $S$: Structuring element


In [ ]:
blackhat_bin_1 = cv2.morphologyEx(im_bin_5, cv2.MORPH_BLACKHAT, se_disk1)
blackhat_bin_2 = cv2.morphologyEx(im_bin_1, cv2.MORPH_BLACKHAT, se_disk3)
blackhat_bin_3 = cv2.morphologyEx(im_bin_6, cv2.MORPH_BLACKHAT, se_diamond1)
blackhat_bin_4 = cv2.morphologyEx(im_bin_2, cv2.MORPH_BLACKHAT, se_square3)

imgs = [im_bin_5, im_bin_1, im_bin_6, im_bin_2]
blackhats = [blackhat_bin_1, blackhat_bin_2, blackhat_bin_3, blackhat_bin_4]
titles = ["im_bin_5 & se_disk1", "im_bin_1 & se_disk3", "im_bin_6 & se_diamond1", "im_bin_2 & se_square3"]

fig, axs = plt.subplots(2, len(blackhats), figsize=(16, 8), layout="compressed")
for col, (img, blackhat_img, title) in enumerate(zip(imgs, blackhats, titles)):
    highlight_idx = blackhat_img > 0
    blackhat_rgb = np.zeros((*blackhat_img.shape, 3), dtype=np.uint8)
    blackhat_rgb[highlight_idx] = [0, 255, 0]

    axs[0, col].imshow(img, cmap="gray")
    axs[0, col].set_title("Original")
    axs[1, col].imshow(blackhat_rgb)
    axs[1, col].set_title(f"Black-Hat: {title}")
plt.show()

#### <a id='toc3_2_7_2_'></a>[Grayscale Black-Hat](#toc0_)
- Highlights small dark regions in grayscale images.
- Helps in detecting dark details and correcting uneven illumination.

$$
(\text{BH}(I))(x) = \left( (I \bullet S) \right)(x) - I(x)
$$

Where:  
- $I$: Input grayscale image  
- $S$: Structuring element

> **Note:** For RGB images, black-hat is usually applied **channel-wise** by processing each channel separately (R, G, B).


In [ ]:
# Apply grayscale Black-Hat (closing - original)
blackhat_gray_1 = cv2.morphologyEx(im_gray_1, cv2.MORPH_BLACKHAT, se_disk1)
blackhat_gray_2 = cv2.morphologyEx(im_gray_2, cv2.MORPH_BLACKHAT, se_disk3)
blackhat_gray_3 = cv2.morphologyEx(im_gray_3, cv2.MORPH_BLACKHAT, se_square3)
blackhat_gray_4 = cv2.morphologyEx(im_gray_5, cv2.MORPH_BLACKHAT, se_disk5)

imgs_gray = [im_gray_1, im_gray_2, im_gray_3, im_gray_5]
blackhats_gray = [blackhat_gray_1, blackhat_gray_2, blackhat_gray_3, blackhat_gray_4]
titles = ["im_gray_1 & se_disk1", "im_gray_2 & se_disk3", "im_gray_3 & se_square3", "im_gray_5 & se_disk5"]

fig, axs = plt.subplots(3, len(blackhats_gray), figsize=(16, 12), layout="compressed")
for col, (img, blackhat, title) in enumerate(zip(imgs_gray, blackhats_gray, titles)):
    diff_img = np.abs(img.astype(np.float32) - blackhat)

    axs[0, col].imshow(img, cmap="gray", vmin=0, vmax=1)
    axs[0, col].set_title("Original Grayscale")
    axs[1, col].imshow(blackhat, cmap="gray", vmin=0, vmax=1)
    axs[1, col].set_title(f"Black-Hat: {title}")
    axs[2, col].imshow(diff_img, cmap="hot", vmin=0, vmax=1)
    axs[2, col].set_title("Absolute difference")
plt.show()